In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import bcolz  # to process the data from Glove File 
import pickle # to dump and load pretrained glove vectors 
import copy   # to make deepcopy of python lists and dictionaries
import operator
import numpy as np
from pandas import DataFrame # to visualize the glove word embeddings in form of DataFrame

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
words = []
idx = 0
word2idx = {}
vectors = bcolz.carray(np.zeros(1), rootdir=f'data/6B.300.dat', mode='w')

with open(f'data/glove.6B.300d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)
    
vectors = bcolz.carray(vectors[1:].reshape((400000, 300)), rootdir=f'data/6B.300.dat', mode='w')
vectors.flush()
pickle.dump(words, open(f'data/6B.300_words.pkl', 'wb'))
pickle.dump(word2idx, open(f'data/6B.300_idx.pkl', 'wb'))

In [4]:
vectors = bcolz.open(f'data/6B.300.dat')[:]
words = pickle.load(open(f'data/6B.300_words.pkl', 'rb'))
word2idx = pickle.load(open(f'data/6B.300_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

In [5]:
glove_dframe = DataFrame(vectors, columns=range(1, 301), index=words)

In [6]:
glove_dframe[100:110]

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
so,-0.245610,0.068010,0.182540,-0.295510,0.018007,0.168110,0.191090,0.063650,0.212430,-2.1165,...,0.095983,-0.347390,0.032640,0.047579,0.222420,-0.231400,0.114310,-0.669140,-0.034753,0.097869
them,-0.205880,0.192570,-0.118270,-0.157120,-0.380160,0.059840,-0.032788,0.174650,0.403970,-1.8908,...,-0.057513,-0.650600,0.028065,-0.135320,0.115990,-0.244490,0.048469,-0.328680,-0.169600,-0.129650
what,-0.200170,0.143020,0.052055,-0.000809,0.017009,0.014899,-0.255240,-0.179070,-0.046713,-2.0547,...,0.045239,-0.352980,0.333500,0.281040,0.203380,-0.478800,-0.039697,0.034939,-0.125990,0.218630
him,-0.158840,0.360950,-0.113370,-0.421430,-0.284790,-0.089365,0.125030,-0.359220,0.027257,-1.6332,...,0.030342,-0.196140,-0.160160,-0.285130,0.277510,-0.143610,0.214780,-0.223810,-0.293430,-0.033526
united,0.056277,-0.165980,0.332410,-0.064785,0.268810,0.108790,-0.807370,0.384470,0.128930,-1.5534,...,0.017244,-0.120950,0.501220,0.161380,0.359710,0.752880,0.479690,-0.274290,-0.171260,-0.803720
during,-0.039635,0.212610,-0.245140,-0.266630,0.189180,0.099316,-0.370910,0.038365,-0.092625,-1.6565,...,-0.148390,-0.020586,-0.697330,0.248950,0.214960,0.351270,-0.146670,-0.142210,-0.464110,-0.159580
before,0.000730,0.109870,0.063975,0.026729,0.100660,0.045820,-0.178650,-0.146560,0.223870,-1.5851,...,0.124490,-0.147260,0.078155,-0.052333,0.012701,0.176650,0.020622,-0.481250,-0.078685,0.200680
may,-0.376040,0.241160,-0.260980,-0.007960,0.219800,0.098515,0.234580,0.137970,0.110260,-1.7503,...,-0.083795,-0.158100,0.325670,-0.429300,0.110570,0.312300,-0.255790,-0.757310,0.097391,-0.081830
since,0.132110,0.009039,0.464310,0.160600,0.289240,-0.014632,-0.309260,-0.102920,0.319730,-2.0459,...,0.104320,-0.210160,0.261070,0.085519,0.171470,0.854160,0.069214,-0.649110,0.410390,0.080164
many,-0.155070,0.284220,0.311910,-0.195960,0.125730,-0.152090,0.145480,0.337110,-0.093931,-1.4419,...,-0.124250,-0.531690,-0.037670,0.150680,-0.139130,-0.088808,-0.227130,-0.545620,-0.147350,-0.314930


In [7]:
sos_index = word2idx['sos']
eos_index = word2idx['eos']
sos_swap_word = words[0]
eos_swap_word = words[1]

In [8]:
words[0], words[sos_index] = words[sos_index], words[0]
words[1], words[eos_index] = words[eos_index], words[1]
word2idx[sos_swap_word], word2idx['sos'] = word2idx['sos'], word2idx[sos_swap_word]
word2idx[eos_swap_word], word2idx['eos'] = word2idx['eos'], word2idx[eos_swap_word]

In [9]:
word2idx = { k : v for k , v in sorted(word2idx.items(), key=operator.itemgetter(1))}

In [10]:
class InputLang:
    def __init__(self, name):
        self.name = name
        self.word2index = { k : v for k , v in sorted(word2idx.items(), key=operator.itemgetter(1))}
        self.word2count = { word : 1 for word in words }
        self.index2word = { i : word for word, i in word2idx.items() }
        self.n_words = 400001

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            

In [11]:
SOS_token = 0
EOS_token = 1


class OutputLang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "sos", 1: "eos"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [12]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [13]:
def readLangs(lang1, lang2,reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/eng-wol.txt', encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
   
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = InputLang("eng")
        output_lang = OutputLang("wol")
    else:
        input_lang = InputLang("eng")
        output_lang = OutputLang("wol")
    

    return input_lang, output_lang, pairs

In [14]:
MAX_LENGTH = 60


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH 


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [15]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'wol', False)
print(random.choice(pairs))

Reading lines...
Read 3687 sentence pairs
Trimmed to 3687 sentence pairs
Counting words...
Counted words:
eng 400100
wol 2218
['one', 'benna']


In [41]:
matrix_len = input_lang.n_words

weights_matrix = np.zeros((matrix_len, 300))
# words_found = 0
miss = []
for i, word in enumerate(input_lang.word2index):
        try: 
            weights_matrix[i] = glove[word]
#             words_found += 1
        except KeyError: miss.append(word)
#             weights_matrix[i] = np.random.normal(scale=0.6, size=(300, ))
   

In [42]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [43]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [44]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [45]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [46]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [47]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [48]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [49]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [50]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [51]:
def evaluateRandomly(encoder, decoder, n=5):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 50
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.01).to(device)

trainIters(encoder1, attn_decoder1,100000, print_every=5000)

23m 39s (- 449m 27s) (5000 5%) 4.0845
50m 52s (- 457m 54s) (10000 10%) 3.9835
72m 49s (- 412m 37s) (15000 15%) 3.9167


In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

In [54]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "hello lamin")
plt.matshow(attentions.numpy())

In [55]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("where does lamin leave")



input = where does lamin leave
output = <EOS>


<ipython-input-55-e847184b7ee5>:9: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''] + input_sentence.split(' ') +
<ipython-input-55-e847184b7ee5>:11: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([''] + output_words)
<ipython-input-55-e847184b7ee5>:17: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [56]:
evaluateAndShowAttention("what is your name ?")


input = what is your name ?
output = <EOS>


<ipython-input-55-e847184b7ee5>:9: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''] + input_sentence.split(' ') +
<ipython-input-55-e847184b7ee5>:11: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([''] + output_words)
<ipython-input-55-e847184b7ee5>:17: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [77]:
# miss = []
# words=''
# for words in (input_lang.word2index):
#     try:evaluateAndShowAttention(words)
#     except KeyError: miss.append(words)


In [ ]:
PATH = "export.pt"

In [46]:
torch.save(AttnDecoderRNN, PATH)


In [51]:
model = torch.load(PATH)
model.eval()

NameError: name 'self' is not defined